## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alofi,NU,-19.0595,-169.9187,78.69,82,44,16.82,light rain
1,1,Carnarvon,AU,-24.8667,113.6333,69.87,78,0,17.27,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,51.46,58,40,31.07,scattered clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,73.36,74,39,11.03,scattered clouds
4,4,Ponta Do Sol,PT,32.6667,-17.1000,63.72,89,33,5.30,scattered clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alofi,NU,-19.0595,-169.9187,78.69,82,44,16.82,light rain
3,3,Puerto Ayora,EC,-0.7393,-90.3518,73.36,74,39,11.03,scattered clouds
5,5,Whithorn,JM,18.2604,-78.0378,82.44,44,85,2.17,overcast clouds
13,13,Atuona,PF,-9.8000,-139.0333,77.54,75,5,13.62,clear sky
14,14,Bathsheba,BB,13.2167,-59.5167,88.41,62,40,13.80,scattered clouds
18,18,Rikitea,PF,-23.1203,-134.9692,72.59,66,6,8.01,clear sky
25,25,Kapaa,US,22.0752,-159.3190,71.19,59,17,6.62,light rain
26,26,San Cristobal,VE,7.7669,-72.2250,75.38,48,85,0.58,overcast clouds
27,27,Garowe,SO,8.4054,48.4845,75.25,52,59,17.43,broken clouds
28,28,Isangel,VU,-19.5500,169.2667,72.82,91,100,19.19,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                248
City                   248
Country                246
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Current Description    248
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
# check that the null rows were dropped
clean_df.count()

City_ID                246
City                   246
Country                246
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Alofi,NU,78.69,light rain,-19.0595,-169.9187,
3,Puerto Ayora,EC,73.36,scattered clouds,-0.7393,-90.3518,
5,Whithorn,JM,82.44,overcast clouds,18.2604,-78.0378,
13,Atuona,PF,77.54,clear sky,-9.8000,-139.0333,
14,Bathsheba,BB,88.41,scattered clouds,13.2167,-59.5167,
18,Rikitea,PF,72.59,clear sky,-23.1203,-134.9692,
25,Kapaa,US,71.19,light rain,22.0752,-159.3190,
26,San Cristobal,VE,75.38,overcast clouds,7.7669,-72.2250,
27,Garowe,SO,75.25,broken clouds,8.4054,48.4845,
28,Isangel,VU,72.82,overcast clouds,-19.5500,169.2667,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found, skipping the city.")
        

Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.
Hotel not found, skipping the city.


In [49]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df = hotel_df.dropna()

City                   225
Country                225
Max Temp               225
Current Description    225
Lat                    225
Lng                    225
Hotel Name             225
dtype: int64

In [51]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))